## References
Our Paper: https://www.cs.cornell.edu/~kb/publications/SIG15ProductNet.pdf

Keras Siamese: https://github.com/harveyslash/Facial-Similarity-with-Siamese-Networks-in-Pytorch/blob/master/Siamese-networks-medium.ipynb

## To Do

1. Embeddings have to be L2 normalized
2. Recheck the Custom loss function

3. Data Loader for DeepFashion:
Reference: https://jovian.ml/gautham20/deepfashion-similar-images-annoy

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.models as models
from torch import optim
import torch.nn.functional as F
from torchsummary import summary

In [2]:
# model = models.resnet18(pretrained=True)
# newmodel = torch.nn.Sequential(*(list(model.children())[:-1]))
# summary(newmodel, (3, 224, 224))

In [3]:
class SiameseNetwork(nn.Module):
    def __init__(self, base_network_name='resnet34', num_classes=46):
        super(SiameseNetwork, self).__init__()
        
        if base_network_name == 'resnet18':
            self.resnet = models.resnet18(pretrained=True)
            self.embd_dim = 512
        elif base_network_name == 'resnet34':
            self.resnet = models.resnet34(pretrained=True)
            self.embd_dim = 512
        elif base_network_name == 'resnet50':
            self.resnet = models.resnet50(pretrained=True)
            self.embd_dim = 2048
        else:
            raise NotImplementedError("Supports only resnet 18, 34 and 50")
        
        self.base_network = torch.nn.Sequential(*(list(self.resnet.children())[:-1]))
            
        self.fc1 = nn.Sequential(
            nn.Linear(self.embd_dim, 256),
            nn.ReLU(inplace=True),

            nn.Linear(256, num_classes))

    def forward_once(self, x):
        output = self.base_network(x)
        embd_output = output.view(output.size()[0], -1)
        class_output = self.fc1(embd_output)
        return embd_output, class_output

    def forward(self, input1, input2):
        embd_output1, class_output1 = self.forward_once(input1)
        embd_output2, class_output2 = self.forward_once(input2)
        return embd_output1, class_output1, embd_output2, class_output2

In [7]:
class Custom_ContrastiveLoss(torch.nn.Module):
    """
    Contrastive loss function.
    Based on: http://yann.lecun.com/exdb/publis/pdf/hadsell-chopra-lecun-06.pdf
    """

    def __init__(self, margin=2.0):
        super(ContrastiveLoss, self).__init__()
        self.margin = margin

    def forward(self, embd_output1, class_output1, embd_output2, class_output2, label, class1, class2):
        euclidean_distance = F.pairwise_distance(embd_output1, embd_output1, keepdim = True)
        loss_contrastive = torch.mean((1-label) * torch.pow(euclidean_distance, 2) +
                                      (label) * torch.pow(torch.clamp(self.margin - euclidean_distance, min=0.0), 2))

        loss_1 = nn.CrossEntropyLoss(class_output1, class1)
        loss_2 = nn.CrossEntropyLoss(class_output2, class2)
        
        return loss_contrastive+loss_1+loss_2